## source: https://www.kaggle.com/iwooloowi/film-awards-imdb

## file: https://www.kaggle.com/iwooloowi/film-awards-imdb/download

In [1]:
# Import our dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import datetime
import re
import csv
import json
import os


In [2]:
# Read in "The IMbd Movies dataset"
# if this is read in as csv, each row is a string and dictionaries are not recognized.
IMDb_movies_df = pd.read_csv("Resources/imdb_main.csv", low_memory=False)
IMDb_movies_df.head()

,imdb_title_id,title,year,genre,duration,country,language,director,writer,production_company,...,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,release_year,release_year.1,genre.1
0,tt0010680,The Sentimental Bloke,1919.0,"Drama, Romance",106,Australia,English,Raymond Longford,"C.J. Dennis, Raymond Longford",Southern Cross Feature Film Company,...,237,NaN,NaN,NaN,NaN,11.0,10.0,2005.0,2005.0,"['Drama', ' Romance']"
1,tt0013579,Ombre ammonitrici,1923.0,"Drama, Fantasy, Horror",90,Germany,German,Arthur Robison,"Arthur Robison, Rudolf Schneider",Pan-Film,...,842,NaN,NaN,NaN,NaN,15.0,16.0,2016.0,2016.0,"['Drama', ' Fantasy', ' Horror']"
2,tt0017938,La glace à trois faces,1983.0,"Drama, Romance",45,France,"None, French",Jean Epstein,"Jean Epstein, Paul Morand",Films Jean Epstein,...,759,NaN,NaN,NaN,NaN,7.0,4.0,1983.0,1983.0,"['Drama', ' Romance']"
3,tt0018054,Il re dei re,1927.0,"Biography, Drama, History",160,USA,English,Cecil B. DeMille,Jeanie Macpherson,DeMille Pictures Corporation,...,1890,$ 2500000,NaN,NaN,NaN,48.0,24.0,2004.0,2004.0,"['Biography', ' Drama', ' History']"
4,tt0018742,Il cameraman,1928.0,"Comedy, Drama, Family",76,USA,English,"Edward Sedgwick, Buster Keaton","Clyde Bruckman, Lew Lipton",Metro-Goldwyn-Mayer (MGM),...,10101,NaN,NaN,$ 1737460,NaN,61.0,49.0,1970.0,1970.0,"['Comedy', ' Drama', ' Family']"


In [3]:
print(IMDb_movies_df.describe())
print(IMDb_movies_df.columns)

               year      duration      avg_vote         votes     metascore  \
count  70529.000000  70529.000000  70529.000000  7.052900e+04  12858.000000   
mean    2002.838223    102.113414      5.790186  1.093330e+04     55.158812   
std       13.745638     21.708421      1.275937  5.836995e+04     17.429233   
min     1919.000000     41.000000      1.000000  9.900000e+01      1.000000   
25%     1994.000000     90.000000      5.000000  2.120000e+02     43.000000   
50%     2007.000000     97.000000      6.000000  5.170000e+02     56.000000   
75%     2014.000000    110.000000      6.700000  2.071000e+03     68.000000   
max     2020.000000    808.000000      9.900000  2.278845e+06    100.000000   

       reviews_from_users  reviews_from_critics  release_year  release_year.1  
count        63624.000000          60360.000000  70529.000000    70529.000000  
mean            50.771376             30.539231   2003.242170     2003.242170  
std            195.785529             63.338663 

In [4]:
IMDb_movies_df.rename(columns={'imdb_title_id':'imdb_id'}, inplace=True)
IMDb_movies_df.head()

,imdb_id,title,year,genre,duration,country,language,director,writer,production_company,...,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,release_year,release_year.1,genre.1
0,tt0010680,The Sentimental Bloke,1919.0,"Drama, Romance",106,Australia,English,Raymond Longford,"C.J. Dennis, Raymond Longford",Southern Cross Feature Film Company,...,237,NaN,NaN,NaN,NaN,11.0,10.0,2005.0,2005.0,"['Drama', ' Romance']"
1,tt0013579,Ombre ammonitrici,1923.0,"Drama, Fantasy, Horror",90,Germany,German,Arthur Robison,"Arthur Robison, Rudolf Schneider",Pan-Film,...,842,NaN,NaN,NaN,NaN,15.0,16.0,2016.0,2016.0,"['Drama', ' Fantasy', ' Horror']"
2,tt0017938,La glace à trois faces,1983.0,"Drama, Romance",45,France,"None, French",Jean Epstein,"Jean Epstein, Paul Morand",Films Jean Epstein,...,759,NaN,NaN,NaN,NaN,7.0,4.0,1983.0,1983.0,"['Drama', ' Romance']"
3,tt0018054,Il re dei re,1927.0,"Biography, Drama, History",160,USA,English,Cecil B. DeMille,Jeanie Macpherson,DeMille Pictures Corporation,...,1890,$ 2500000,NaN,NaN,NaN,48.0,24.0,2004.0,2004.0,"['Biography', ' Drama', ' History']"
4,tt0018742,Il cameraman,1928.0,"Comedy, Drama, Family",76,USA,English,"Edward Sedgwick, Buster Keaton","Clyde Bruckman, Lew Lipton",Metro-Goldwyn-Mayer (MGM),...,10101,NaN,NaN,$ 1737460,NaN,61.0,49.0,1970.0,1970.0,"['Comedy', ' Drama', ' Family']"


In [5]:
awards_df = pd.read_csv("Resources/awards.csv", low_memory=False)
awards_df.rename(columns={'const':'imdb_id'}, inplace=True)
awards_df.head()

,eventId,eventName,awardName,year,occurrence,winAnnouncementTime,categoryName,nomeneeNote,name,originalName,...,episodeNames,characterNames,isWinner,isPrimary,isSecondary,isPerson,isTitle,isCompany,imdb_id,notes
0,ev0000263,French Syndicate of Cinema Critics,Critics Award,1992,1,NaN,Best Film,NaN,La belle noiseuse,NaN,...,NaN,NaN,True,True,False,False,True,False,tt0101428,NaN
1,ev0000263,French Syndicate of Cinema Critics,Critics Award,1992,1,NaN,Best Film,NaN,Jacques Rivette,NaN,...,NaN,NaN,True,False,True,True,False,False,nm0729626,NaN
2,ev0000263,French Syndicate of Cinema Critics,Critics Award,1992,1,NaN,Best Foreign Film,NaN,The Double Life of Véronique,La double vie de Véronique,...,NaN,NaN,True,True,False,False,True,False,tt0101765,Poland.
3,ev0000263,French Syndicate of Cinema Critics,Critics Award,1992,1,NaN,Best Foreign Film,NaN,Krzysztof Kieslowski,NaN,...,NaN,NaN,True,False,True,True,False,False,nm0001425,Poland.
4,ev0000263,French Syndicate of Cinema Critics,Critics Award,1992,1,NaN,Best Short,NaN,Le petit chat est mort,NaN,...,NaN,NaN,True,True,False,False,True,False,tt0195135,NaN


In [7]:
len(awards_df)

2158525

In [8]:
awards_df=awards_df[awards_df["year"] > 1969]

In [9]:
len(awards_df)

2104294

In [10]:
awards_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2104294 entries, 0 to 2158524
Data columns (total 21 columns):
 #   Column               Dtype  
---  ------               -----  
 0   eventId              object 
 1   eventName            object 
 2   awardName            object 
 3   year                 int64  
 4   occurrence           int64  
 5   winAnnouncementTime  float64
 6   categoryName         object 
 7   nomeneeNote          object 
 8   name                 object 
 9   originalName         object 
 10  songNames            object 
 11  episodeNames         object 
 12  characterNames       object 
 13  isWinner             object 
 14  isPrimary            object 
 15  isSecondary          object 
 16  isPerson             bool   
 17  isTitle              bool   
 18  isCompany            bool   
 19  imdb_id              object 
 20  notes                object 
dtypes: bool(3), float64(1), int64(2), object(15)
memory usage: 311.1+ MB


In [11]:
cols=awards_df.columns

In [12]:
for col in cols:
    print(f'\n datatypes in {col} column: {awards_df[col].apply(type).value_counts()}')


 datatypes in eventId column: <class 'str'>    2104294
Name: eventId, dtype: int64

 datatypes in eventName column: <class 'str'>    2104294
Name: eventName, dtype: int64

 datatypes in awardName column: <class 'str'>      2103551
<class 'float'>        743
Name: awardName, dtype: int64

 datatypes in year column: <class 'int'>    2104294
Name: year, dtype: int64

 datatypes in occurrence column: <class 'int'>    2104294
Name: occurrence, dtype: int64

 datatypes in winAnnouncementTime column: <class 'float'>    2104294
Name: winAnnouncementTime, dtype: int64

 datatypes in categoryName column: <class 'str'>      1865098
<class 'float'>     239196
Name: categoryName, dtype: int64

 datatypes in nomeneeNote column: <class 'float'>    1747661
<class 'str'>       356633
Name: nomeneeNote, dtype: int64

 datatypes in name column: <class 'str'>      2099853
<class 'float'>       4441
Name: name, dtype: int64

 datatypes in originalName column: <class 'float'>    1922468
<class 'str'>      

In [13]:
for col in cols:
    print(f'\n value_counts in {col} column: {awards_df[col].value_counts(dropna=False)}')


 value_counts in eventId column: ev0000223    79023
ev0000206    49369
ev0000123    27538
ev0000493    26331
ev0002704    25255
             ...  
ev0004121        1
ev0005188        1
ev0005543        1
ev0015612        1
ev0004605        1
Name: eventId, Length: 6640, dtype: int64

 value_counts in eventName column: Primetime Emmy Awards                       79023
Daytime Emmy Awards                         49369
BAFTA Awards                                27538
News & Documentary Emmy Awards              26331
Online Film & Television Association        25255
                                            ...  
Japanese Cultural Ministery Movie Awards        1
DY Patil Annual Achiever's Awards               1
Lavazza Italian Film Festival                   1
FDF - Festival dokumentarnega filma             1
Screen Student Film Festival, US                1
Name: eventName, Length: 6640, dtype: int64

 value_counts in awardName column: Primetime Emmy                       78915
Daytim